In [ ]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from other_repos.pyDMS.pyDMS.pyDMS import *
import time

In [ ]:
x = 70
y = 44
year = 2019
bands = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/force/output/Guzinski/{year}/tiles/X00{x}_Y00{y}/', '.tif') if '_20190617' in file]
bands = [bands[i] for i in [0,1,6,3,4,5,2,7,8]]

In [ ]:
ds = gdal.Open(f'/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/SLOPE/SLOPE_X00{x}_Y00{y}.tif')
slope = ds.GetRasterBand(1).ReadAsArray()
ds = gdal.Open(f'/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/ASPECT/ASPECT_X00{x}_Y00{y}.tif')
aspect = ds.GetRasterBand(1).ReadAsArray()

In [ ]:
arrs = []
for band in bands:
    ds = gdal.Open(band)
    arrs.append(ds.GetRasterBand(1).ReadAsArray())
arrs.append(slope)
arrs.append(aspect)
arr = np.dstack(arrs)
mask = np.logical_and(arr[:,:,9] < 0.000000001, arr[:,:,10] == 180)
masked_arr = np.where(mask[:,:,None],np.nan, arr)
npTOdisk(masked_arr, reference_path=bands[0], outPath='/data/Aldhani/eoagritwin/et/Auxiliary/trash/S2_stack_test.tif', bands=len(bands))

In [ ]:
path = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/minVZA/2019/Daily_LST_minVZA_2019_June.tif'
ds = gdal.Open(path)
arr = ds.GetRasterBand(17).ReadAsArray()
npTOdisk(arr, path, '/data/Aldhani/eoagritwin/et/Auxiliary/trash/LST.tif', bands=1)

In [ ]:
ds = gdal.Open('/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/ef7ec2b58193b34b417ff31ee21459a5a343a0e02e2ca62f416e035eca63234e/minVZA/2019/July/07/withoutLSTmask/S2notMasked/Values/minVZA_2019_July_07_mvwin15_cv0_regrat25_S2notMasked_withoutLSTmask.tif')
ras = ds.GetRasterBand(1).ReadAsArray()
plotter(ras)

In [ ]:
ds = gdal.Open('/data/Aldhani/eoagritwin/et/Auxiliary/trash/sharpi_test.tif')
ras = ds.GetRasterBand(1).ReadAsArray()
np.unique(ras)

In [ ]:
highResFilename = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/S2_stack_test.tif'
lowResFilename = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/LST.tif'
lowResMaskFilename = r""
outputFilename = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/sharpi_test.tif'

##########################################################################################

if __name__ == "__main__":

    useDecisionTree = True

    commonOpts = {"highResFiles":               [highResFilename],
                  "lowResFiles":                [lowResFilename],
                  "lowResQualityFiles":         [lowResMaskFilename],
                  "lowResGoodQualityFlags":     [255],
                  "cvHomogeneityThreshold":     0,
                  "movingWindowSize":           15,
                  "disaggregatingTemperature":  True}
    dtOpts =     {"perLeafLinearRegression":    True,
                  "linearRegressionExtrapolationRatio": 0.25}
    sknnOpts =   {'hidden_layer_sizes':         (10,),
                  'activation':                 'tanh'}
    nnOpts =     {"regressionType":             REG_sklearn_ann,
                  "regressorOpt":               sknnOpts}

    start_time = time.time()

    if useDecisionTree:
        opts = commonOpts.copy()
        opts.update(dtOpts)
        disaggregator = DecisionTreeSharpener(**opts)
    else:
        opts = commonOpts.copy()
        opts.update(nnOpts)
        disaggregator = NeuralNetworkSharpener(**opts)

    print("Training regressor...")
    disaggregator.trainSharpener()
    print("Sharpening...")
    downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
    print("Residual analysis...")
    residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                   lowResMaskFilename,
                                                                   doCorrection=True)
    print("Saving output...")
    highResFile = gdal.Open(highResFilename)
    if correctedImage is not None:
        outImage = correctedImage
    else:
        outImage = downscaledFile
    # outData = utils.binomialSmoother(outData)
    outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                            outImage.GetGeoTransform(),
                            outImage.GetProjection(),
                            outputFilename)
    residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                 residualImage.GetGeoTransform(),
                                 residualImage.GetProjection(),
                                 os.path.splitext(outputFilename)[0] + "_residual" +
                                 os.path.splitext(outputFilename)[1])

    outFile = None
    residualFile = None
    downsaceldFile = None
    highResFile = None

    print(time.time() - start_time, "seconds")

In [ ]:
## make vrts from single FORCE tile tests

base_path = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/'

folders = ['S2only', 'all_preds']

comps = ['maxLST', 'minVZA']

days = [f'July_{i:02d}' for i in range(1,10,1)]

masks = ['S2Masked_withLSTmask', 'S2Masked_withoutLSTmask', 'S2notMasked_withLSTmask', 'S2notMasked_withoutLSTmask']

for path in [f'{base_path}vrts/{day}' for day in days]:
    os.makedirs(path, exist_ok=True)

In [ ]:
for folder in folders:
    for comp in comps:
        for day in days:
            for mask in masks:
                file_list = [file for file in getFilelist(base_path + folder, '.tif', deep=True) if comp in file and day in file and mask in file and 'resid' not in file]
                outpath = base_path + f'vrts/{day}/{folder}_{comp}_{day}_{mask}.vrt'
                gdal.BuildVRT(outpath, file_list, separate=False)
                convertVRTpathsTOrelative(outpath)
                ds = gdal.Open(outpath)
                arr = ds.GetRasterBand(1).ReadAsArray()
                print(f'{day}/{folder}_{comp}_{day}_{mask}:  {np.nanmax(arr)}')


In [11]:
# export statistics

results = []

for folder in folders:
    for comp in comps:
        for day in days:
            for mask in masks:
                file_list = [file for file in getFilelist(base_path + folder, '.tif', deep=True) if comp in file and day in file and mask in file and 'resid' not in file]
                outpath = base_path + f'vrts/{day}/{folder}_{comp}_{day}_{mask}.vrt'

                # open dataset
                ds = gdal.Open(outpath)
                arr = ds.GetRasterBand(1).ReadAsArray()
                
                maxLST = np.nanmax(arr)
                minLST = np.nanmin(arr)
                p25, p50, p75 = np.nanpercentile(arr, [25, 50, 75])

                # Append result
                results.append({
                    'folder': folder,
                    'comp': comp,
                    'day': day,
                    'mask': mask,
                    'maxLST': maxLST,
                    'minLST': minLST,
                    '25thLST': p25,
                    'medianLST': p50,
                    '75thLST': p75
                })


# convert and export
df = pd.DataFrame(results)
df.to_csv('../csvs/basic_stats_vrt_44.csv', index=False)


'/home/potzschf/repos/evapo_scripts/Guzinski/ipynb'